In [56]:
import yfinance as yf
import numpy as np
from collections import deque
import pandas as pd
from datetime import datetime

In [57]:
ticker = "MSFT"  # 005930.KS
df = yf.download(ticker, start="2015-01-01")

df = df.reset_index()
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

date_array = df['Date'].values
open_array = df['Open'].values
high_array = df['High'].values
low_array = df['Low'].values
close_array = df['Close'].values

df

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-01-02,46.660000,47.419998,46.540001,46.759998,40.452732,27913900
1,2015-01-05,46.369999,46.730000,46.250000,46.330002,40.080734,39673900
2,2015-01-06,46.380001,46.750000,45.540001,45.650002,39.492470,36447900
3,2015-01-07,45.980000,46.459999,45.490002,46.230000,39.994232,29114100
4,2015-01-08,46.750000,47.750000,46.720001,47.590000,41.170780,29645200
...,...,...,...,...,...,...,...
2287,2024-02-05,409.899994,411.160004,403.989990,405.649994,405.649994,25352300
2288,2024-02-06,405.880005,407.970001,402.910004,405.489990,405.489990,18382600
2289,2024-02-07,407.440002,414.299988,407.399994,414.049988,414.049988,22340500
2290,2024-02-08,414.049988,415.559998,412.529999,414.109985,414.109985,21225300


In [58]:
class Zigzag:
    def __init__(self, pivot_range, fluctuation_rate):
        self.pivot_range = pivot_range

        self.fluctuation_rate = fluctuation_rate

        self.buffer_size = pivot_range * 2 + 1

        self.opens = deque(maxlen = self.buffer_size)
        self.highs = deque(maxlen = self.buffer_size)
        self.lows = deque(maxlen = self.buffer_size)
        self.closes = deque(maxlen = self.buffer_size)
        self.dates = deque(maxlen = self.buffer_size)

        self.last_pivot = [None, None, None, None, None] # (date, is high pivot?, pivot price, is this pivot confirmed to zigzag pivot?, is pivot on bullish candle?)


    def update_zigzag_pivot(self, open, high, low, close, date):
        self.zigzag_pivot = []

        self.opens.append(open)
        self.highs.append(high)
        self.lows.append(low)
        self.closes.append(close)
        self.dates.append(date)

        if len(self.opens) == self.buffer_size:
            pivot_list = []
            pivot_high = self.is_pivot(self.highs, self.pivot_range, True)
            pivot_low = self.is_pivot(self.lows, self.pivot_range, False)

            bullish = self.opens[self.pivot_range] < self.closes[self.pivot_range]

            if pivot_high is not None:
                pivot_list.append([self.dates[self.pivot_range], True, pivot_high, False, bullish])
            if pivot_low is not None:
                pivot_list.append([self.dates[self.pivot_range], False, pivot_low, False, bullish])

            if len(pivot_list) == 2 and bullish:
                pivot_list = [pivot_list[1], pivot_list[0]]

            self.update_last_pivot(pivot_list, high, low, date)

            if self.last_pivot != [None, None, None, None, None]:
                self.last_pivot_to_zigzag(True)

        return self.zigzag_pivot


    def is_pivot(self, array, candle, is_high):
        if is_high and candle == np.argmax(array):
            return array[candle]

        elif is_high == False and candle == np.argmin(array):
            return array[candle]

        return None

    def is_valid_reversals(self, pivot_price, last_pivot_price, last_pivot_type):
        change_percentage = (pivot_price - last_pivot_price) / last_pivot_price * 100
        is_valid_reversals = (not last_pivot_type and change_percentage >= self.fluctuation_rate) or (last_pivot_type and change_percentage <= -self.fluctuation_rate)
        return is_valid_reversals


    def update_last_pivot(self, pivot_list, high, low, date):
        for pivot in pivot_list:
            if self.last_pivot == [None, None, None, None, None]:
                self.last_pivot = pivot
                self.last_pivot_to_zigzag(False)

            else:
                if self.last_pivot[1] == pivot[1]:
                    if (self.last_pivot[1] and self.last_pivot[2] < pivot[2]) or (not self.last_pivot[1] and self.last_pivot[2] > pivot[2]):
                        self.last_pivot = pivot
                        self.last_pivot_to_zigzag(False)

                else:
                    reversals = self.is_valid_reversals(pivot[2], self.last_pivot[2], self.last_pivot[1])
                    if reversals:
                        self.last_pivot = pivot
                        self.last_pivot_to_zigzag(False)


    def last_pivot_to_zigzag(self, one_candle):
        if self.last_pivot[3] == True:
            return

        if one_candle:
            start_idx = self.buffer_size - self.pivot_range - 1
            end_idx = self.buffer_size
            self.is_zigzag_pivot(start_idx, end_idx)

        else:
            start = 0 if (self.last_pivot[4] and self.last_pivot[1] == False) or (self.last_pivot[4] == False and self.last_pivot[1]) else 1
            for i in range(start,self.pivot_range):
                if self.last_pivot[3] == False:
                    start_idx = i
                    end_idx = i + self.pivot_range + 1
                    self.is_zigzag_pivot(start_idx, end_idx)


    def is_zigzag_pivot(self, start_idx, end_idx):
        if self.last_pivot[1]:
            lows_list = list(self.lows)
            pivot = self.is_pivot(lows_list[start_idx:end_idx], self.pivot_range, False)

        else:
            highs_list = list(self.highs)
            pivot = self.is_pivot(highs_list[start_idx:end_idx], self.pivot_range, True)

        if pivot is not None:
            reversals = self.is_valid_reversals(pivot, self.last_pivot[2], self.last_pivot[1])
            if reversals:
                self.zigzag_pivot.append([self.last_pivot[0], self.last_pivot[2], self.dates[-1]]) # zigzag pivot date, zigzag pivot price, zigzag pivot confirmed date
                self.last_pivot[3] = True

In [59]:
zigzag_array = []
range5_fluctuation5 = Zigzag(3, 3)
for i in range(len(date_array)):
    date = date_array[i]
    open = open_array[i]
    high = high_array[i]
    low = low_array[i]
    close = close_array[i]

    zigzag_pivot = range5_fluctuation5.update_zigzag_pivot(open, high, low, close, date)
    if zigzag_pivot:
        zigzag_array.extend(zigzag_pivot)


zigzag_array

[['2015-01-07', 45.4900016784668, '2015-01-12'],
 ['2015-01-13', 47.90999984741211, '2015-01-16'],
 ['2015-01-16', 45.16999816894531, '2015-01-22'],
 ['2015-01-23', 47.38999938964844, '2015-01-28'],
 ['2015-02-02', 40.22999954223633, '2015-02-05'],
 ['2015-02-24', 44.29999923706055, '2015-03-04'],
 ['2015-03-13', 40.61000061035156, '2015-03-18'],
 ['2015-03-24', 43.16999816894531, '2015-03-27'],
 ['2015-04-02', 40.119998931884766, '2015-04-08'],
 ['2015-04-30', 49.540000915527344, '2015-05-05'],
 ['2015-05-06', 46.02000045776367, '2015-05-13'],
 ['2015-05-15', 48.90999984741211, '2015-05-20'],
 ['2015-05-26', 46.189998626708984, '2015-05-29'],
 ['2015-05-28', 48.02000045776367, '2015-06-04'],
 ['2015-06-09', 45.459999084472656, '2015-06-12'],
 ['2015-06-11', 46.91999816894531, '2015-06-16'],
 ['2015-06-15', 45.02000045776367, '2015-06-18'],
 ['2015-06-19', 46.83000183105469, '2015-06-26'],
 ['2015-07-07', 43.31999969482422, '2015-07-10'],
 ['2015-07-21', 47.33000183105469, '2015-07-24'

In [60]:
class HarmornicPattern(Zigzag):
    def __init__(self, pivot_range, fluctuation_rate, tolerance_percentage):
        super().__init__(pivot_range, fluctuation_rate)

        self.tolerance_percentage = tolerance_percentage

        self.ABCD_array = deque(maxlen = 4)
        self.XABCD_array = deque(maxlen = 5)


    def update_harmonic_pattern(self, open, high, low, close, date):
        self.harmornic_pattern = [] # pattern name, dates, prices, is bullish pattern?, confirmed date
        zigzag_pivot = self.update_zigzag_pivot(open, high, low, close, date)

        if zigzag_pivot:
            self.ABCD_array.extend(zigzag_pivot)
            self.XABCD_array.extend(zigzag_pivot)


        if len(self.ABCD_array) == 4 and self.ABCD_array[-1][-1] == date:
            A, B, C, D = self.ABCD_array
            bullish, pattern_name = self.is_ABCD_pattern(A[1], B[1], C[1], D[1])
            if pattern_name is not None:
                self.harmornic_pattern.append([pattern_name,
                                         [A[0], B[0], C[0], D[0]],
                                         [A[1], B[1], C[1], D[1]],
                                         bullish,
                                         D[2]])

        if len(self.XABCD_array) == 5 and self.XABCD_array[-1][-1] == date:
            X, A, B, C, D = self.XABCD_array
            bullish, pattern_name = self.is_XABCD_pattern(X[1], A[1], B[1], C[1], D[1], self.tolerance_percentage)

            if pattern_name is not None:
                self.harmornic_pattern.append([pattern_name,
                                         [X[0], A[0], B[0], C[0], D[0]],
                                         [X[1], A[1], B[1], C[1], D[1]],
                                         bullish,
                                         D[2]])

        return self.harmornic_pattern


    def is_ratio_valid(self, ratio, true_ratio):
        lower_bound = true_ratio * (1 - self.tolerance_percentage / 100)
        upper_bound = true_ratio * (1 + self.tolerance_percentage / 100)

        if lower_bound <= ratio <= upper_bound:
            return True
        else:
            return False


    def is_ABCD_pattern(self, A, B, C, D):

        if A - B != 0 and B - C !=0:
            ABC = abs((C - B)/(A - B))
            BCD = abs((D - C)/(B - C))

            if (self.is_ratio_valid(ABC, 0.618) or self.is_ratio_valid(ABC, 0.786)) and (self.is_ratio_valid(BCD,1.272) or self.is_ratio_valid(BCD,1.618)):
                if A < B:
                    return 0, 'ABCD'
                else:
                    return 1, 'ABCD'
        return None, None


    def is_XABCD_pattern(self, X, A, B, C, D, tolerance_percentage):

        XAB = abs((B - A)/(X - A))
        ABC = abs((C - B)/(A - B))
        BCD = abs((D - C)/(B - C))
        XAD = abs((D - A)/(X - A))
        if X - A != 0 and A - B !=0 and B - C != 0 and X - C != 0:
            if ((self.is_ratio_valid(XAB, 0.382) or self.is_ratio_valid(XAB, 0.5)) \
                    and (self.is_ratio_valid(ABC, 0.382) or self.is_ratio_valid(ABC, 0.886)) \
                    and (self.is_ratio_valid(BCD, 1.618) or self.is_ratio_valid(BCD, 2.618)) \
                    and (self.is_ratio_valid(XAD, 0.886))):
                return (1, 'Bat') if X < A else (0, 'Bat')

            if ((self.is_ratio_valid(XAB, 0.618)) \
                    and (self.is_ratio_valid(ABC, 0.382) or self.is_ratio_valid(ABC, 0.886)) \
                    and (self.is_ratio_valid(BCD, 1.272) or self.is_ratio_valid(BCD, 1.618)) \
                    and (self.is_ratio_valid(XAD, 0.786))):
                return (1, 'Gartley') if X < A else (0, 'Gartley')

            if ((self.is_ratio_valid(XAB, 0.382) or self.is_ratio_valid(XAB, 0.618)) \
                    and (self.is_ratio_valid(ABC, 0.382) or self.is_ratio_valid(ABC, 0.886)) \
                    and (self.is_ratio_valid(BCD, 2.242) or self.is_ratio_valid(BCD, 3.618)) \
                    and (self.is_ratio_valid(XAD, 1.618))):
                return (1, 'Crab') if X < A else (0, 'Crab')

            if ((self.is_ratio_valid(XAB, 0.786)) \
                    and (self.is_ratio_valid(ABC, 0.382) or self.is_ratio_valid(ABC, 0.886)) \
                    and (self.is_ratio_valid(BCD, 1.618) or self.is_ratio_valid(BCD, 2.242)) \
                    and (self.is_ratio_valid(XAD, 1.272) or self.is_ratio_valid(XAD, 1.618))):
                return (1, 'Butterfly') if X < A else (0, 'Butterfly')
        return None, None

In [61]:
harmornic_pattern_array = []
range5_fluctuation5_additional_range2 = HarmornicPattern(3, 3, 12)
for i in range(len(date_array)):
    date = date_array[i]
    open = open_array[i]
    high = high_array[i]
    low = low_array[i]
    close = close_array[i]

    harmornic_pattern = range5_fluctuation5_additional_range2.update_harmonic_pattern(open, high, low, close, date)

    if harmornic_pattern:
        harmornic_pattern_array.extend(harmornic_pattern)

result = pd.DataFrame(harmornic_pattern_array, columns = ['pattern_name', 'date', 'price', 'bullish', 'confirmed_date'])
result

,pattern_name,date,price,bullish,confirmed_date
0,ABCD,"[2015-02-24, 2015-03-13, 2015-03-24, 2015-04-02]","[44.29999923706055, 40.61000061035156, 43.1699...",1,2015-04-08
1,ABCD,"[2015-05-15, 2015-05-26, 2015-05-28, 2015-06-09]","[48.90999984741211, 46.189998626708984, 48.020...",1,2015-06-12
2,ABCD,"[2015-05-28, 2015-06-09, 2015-06-11, 2015-06-15]","[48.02000045776367, 45.459999084472656, 46.919...",1,2015-06-18
3,ABCD,"[2015-07-07, 2015-07-21, 2015-07-28, 2015-08-05]","[43.31999969482422, 47.33000183105469, 44.7900...",0,2015-08-10
4,ABCD,"[2015-08-24, 2015-08-28, 2015-09-01, 2015-09-17]","[39.720001220703125, 44.150001525878906, 41.65...",0,2015-09-22
5,ABCD,"[2015-11-13, 2015-12-04, 2015-12-14, 2015-12-17]","[52.529998779296875, 56.22999954223633, 53.680...",0,2015-12-22
6,ABCD,"[2016-03-21, 2016-04-04, 2016-04-12, 2016-04-19]","[52.93000030517578, 55.65999984741211, 53.7599...",0,2016-04-22
7,ABCD,"[2016-04-29, 2016-05-16, 2016-05-19, 2016-05-31]","[49.349998474121094, 51.959999084472656, 49.81...",0,2016-06-10
8,ABCD,"[2016-05-31, 2016-06-13, 2016-06-23, 2016-06-27]","[53.0, 49.060001373291016, 52.060001373291016,...",1,2016-06-30
9,ABCD,"[2016-10-25, 2016-11-04, 2016-11-08, 2016-11-14]","[61.369998931884766, 58.52000045776367, 60.779...",1,2016-11-17


In [62]:
result['pattern_name'].value_counts()

ABCD         25
Gartley       2
Butterfly     2
Bat           2
Name: pattern_name, dtype: int64

In [63]:
def back_test(stock, result, risk_reward_ratio, bull_only):

    result_copy = result.copy()
    result_copy['profit(%)'] = 0
    result_copy['holding_days'] = 0
    result_copy['buy_date'] = pd.NaT
    result_copy['sell_date'] = pd.NaT

    for i in range(len(result)):
        last_date = result.loc[i, 'date'][-1]
        confirmed_date = result.loc[i, 'confirmed_date']
        bullish = result.loc[i, 'bullish']

        if bull_only and not bullish: # if bull_only is True, consider only bullish patterns
            continue

        entry_price = stock.loc[stock['Date'] == confirmed_date, 'Close'].item()

        if bullish:
            stop_loss_price = stock.loc[stock['Date'] == last_date, 'Low'].item()
            risk_amount = abs(entry_price - stop_loss_price)
            profit_amount = risk_amount * risk_reward_ratio
            take_profit_price = entry_price + profit_amount
        else:
            stop_loss_price = stock.loc[stock['Date'] == last_date, 'High'].item()
            risk_amount = abs(entry_price - stop_loss_price)
            profit_amount = risk_amount * risk_reward_ratio
            take_profit_price = entry_price - profit_amount


        entry_idx = stock.index[stock['Date'] == confirmed_date].item()

        profit, sold_date = is_sold(stock, entry_idx+1, len(stock), entry_price, bullish, stop_loss_price, take_profit_price, risk_amount, profit_amount)
        if profit is not None and profit:
            result_copy.loc[i, 'profit(%)'] = profit
            result_copy.loc[i, 'buy_date'] = confirmed_date
            result_copy.loc[i, 'sell_date'] = sold_date
            delta = datetime.strptime(sold_date, '%Y-%m-%d') - datetime.strptime(confirmed_date, '%Y-%m-%d')
            result_copy.loc[i, 'holding_days'] = delta.days

    return result_copy


def is_sold(stock, start, end, entry_price, bullish, stop_loss_price, take_profit_price, risk_amount, profit_amount):
    for idx in range(start, end):
        low = stock.loc[idx, 'Low']
        high = stock.loc[idx, 'High']
        date = stock.loc[idx, 'Date']

        if (bullish and low <= stop_loss_price and high >= take_profit_price) or \
        (not bullish and high >= stop_loss_price and low <= take_profit_price):
            return None, None

        elif (bullish and low <= stop_loss_price) or (not bullish and high >= stop_loss_price):
            return round(-risk_amount / entry_price * 100, 2), date

        elif (bullish and high >= take_profit_price) or (not bullish and low <= take_profit_price):
            return round(profit_amount / entry_price * 100, 2), date

    return False, None

In [64]:
report = back_test(df, result, 3, True)
report

,pattern_name,date,price,bullish,confirmed_date,profit(%),holding_days,buy_date,sell_date
0,ABCD,"[2015-02-24, 2015-03-13, 2015-03-24, 2015-04-02]","[44.29999923706055, 40.61000061035156, 43.1699...",1,2015-04-08,9.42,16,2015-04-08,2015-04-24
1,ABCD,"[2015-05-15, 2015-05-26, 2015-05-28, 2015-06-09]","[48.90999984741211, 46.189998626708984, 48.020...",1,2015-06-12,-1.11,3,2015-06-12,2015-06-15
2,ABCD,"[2015-05-28, 2015-06-09, 2015-06-11, 2015-06-15]","[48.02000045776367, 45.459999084472656, 46.919...",1,2015-06-18,-3.64,11,2015-06-18,2015-06-29
3,ABCD,"[2015-07-07, 2015-07-21, 2015-07-28, 2015-08-05]","[43.31999969482422, 47.33000183105469, 44.7900...",0,2015-08-10,0.00,0,NaT,NaT
4,ABCD,"[2015-08-24, 2015-08-28, 2015-09-01, 2015-09-17]","[39.720001220703125, 44.150001525878906, 41.65...",0,2015-09-22,0.00,0,NaT,NaT
5,ABCD,"[2015-11-13, 2015-12-04, 2015-12-14, 2015-12-17]","[52.529998779296875, 56.22999954223633, 53.680...",0,2015-12-22,0.00,0,NaT,NaT
6,ABCD,"[2016-03-21, 2016-04-04, 2016-04-12, 2016-04-19]","[52.93000030517578, 55.65999984741211, 53.7599...",0,2016-04-22,0.00,0,NaT,NaT
7,ABCD,"[2016-04-29, 2016-05-16, 2016-05-19, 2016-05-31]","[49.349998474121094, 51.959999084472656, 49.81...",0,2016-06-10,0.00,0,NaT,NaT
8,ABCD,"[2016-05-31, 2016-06-13, 2016-06-23, 2016-06-27]","[53.0, 49.060001373291016, 52.060001373291016,...",1,2016-06-30,18.35,116,2016-06-30,2016-10-24
9,ABCD,"[2016-10-25, 2016-11-04, 2016-11-08, 2016-11-14]","[61.369998931884766, 58.52000045776367, 60.779...",1,2016-11-17,16.62,195,2016-11-17,2017-05-31


In [65]:
filtered_report=report.copy()
filtered_report.dropna(subset=['sell_date'], inplace=True)

filtered_report['buy_date'] = pd.to_datetime(filtered_report['buy_date'], format='%Y-%m-%d')
filtered_report['sell_date'] = pd.to_datetime(filtered_report['sell_date'], format='%Y-%m-%d')

filtered_report = filtered_report.sort_values(by='buy_date').reset_index(drop=True)

valid_indices = []
last_sell_date = None

for i in range(len(filtered_report)):
    buy_date = filtered_report.loc[i, 'buy_date']
    if last_sell_date is None or last_sell_date <= buy_date:
        valid_indices.append(i)
        last_sell_date = filtered_report.loc[i, 'sell_date']

filtered_report = filtered_report.loc[valid_indices]

filtered_report


,pattern_name,date,price,bullish,confirmed_date,profit(%),holding_days,buy_date,sell_date
0,ABCD,"[2015-02-24, 2015-03-13, 2015-03-24, 2015-04-02]","[44.29999923706055, 40.61000061035156, 43.1699...",1,2015-04-08,9.42,16,2015-04-08,2015-04-24
1,ABCD,"[2015-05-15, 2015-05-26, 2015-05-28, 2015-06-09]","[48.90999984741211, 46.189998626708984, 48.020...",1,2015-06-12,-1.11,3,2015-06-12,2015-06-15
2,ABCD,"[2015-05-28, 2015-06-09, 2015-06-11, 2015-06-15]","[48.02000045776367, 45.459999084472656, 46.919...",1,2015-06-18,-3.64,11,2015-06-18,2015-06-29
3,ABCD,"[2016-05-31, 2016-06-13, 2016-06-23, 2016-06-27]","[53.0, 49.060001373291016, 52.060001373291016,...",1,2016-06-30,18.35,116,2016-06-30,2016-10-24
4,ABCD,"[2016-10-25, 2016-11-04, 2016-11-08, 2016-11-14]","[61.369998931884766, 58.52000045776367, 60.779...",1,2016-11-17,16.62,195,2016-11-17,2017-05-31
6,ABCD,"[2017-10-27, 2017-11-17, 2017-11-28, 2017-12-04]","[86.19999694824219, 82.23999786376953, 85.0599...",1,2017-12-08,12.33,52,2017-12-08,2018-01-29
7,ABCD,"[2018-10-03, 2018-10-11, 2018-10-17, 2018-10-30]","[116.18000030517578, 104.19999694824219, 111.8...",1,2018-11-06,-7.06,14,2018-11-06,2018-11-20
8,Bat,"[2019-03-27, 2019-04-25, 2019-05-13, 2019-05-1...","[115.5199966430664, 131.3699951171875, 123.040...",1,2019-06-06,20.68,190,2019-06-06,2019-12-13
9,ABCD,"[2020-02-19, 2020-02-28, 2020-03-03, 2020-03-16]","[188.17999267578125, 152.0, 175.0, 135.0]",1,2020-03-19,-5.40,4,2020-03-19,2020-03-23
10,ABCD,"[2020-07-09, 2020-07-17, 2020-07-21, 2020-07-24]","[216.3800048828125, 201.38999938964844, 213.94...",1,2020-07-31,10.98,27,2020-07-31,2020-08-27


In [66]:
start_price = df['Close'][0]
end_price = df['Close'][len(df)-1]

buyhold_profit = ((end_price - start_price) / start_price) * 100
print(f'buy&hold return: {round(buyhold_profit,2)}%')

start_day = df['Date'][0]
end_day = df['Date'][len(df)-1]
delta = datetime.strptime(end_day, '%Y-%m-%d') - datetime.strptime(start_day, '%Y-%m-%d')

total_return = 1
for r in filtered_report['profit(%)']:
    total_return *= (1 + r / 100)

total_return_percentage = (total_return - 1) * 100
print(f'strategy return: {round(total_return_percentage, 2)}%')


total_days= filtered_report['holding_days'].sum()
print(f'buy&hold holding period: {delta.days} days')
print(f'strategy holding period: {total_days} days')


buy_hold_daily_return = (((1 + buyhold_profit / 100) ** (1 / delta.days)) - 1) * 100
strategy_daily_return = (((1 + total_return_percentage / 100) ** (1 / total_days)) - 1) * 100

print(f'buy&hold daily return: {round(buy_hold_daily_return, 4)}%')
print(f'strategy daily return: {round(strategy_daily_return, 4)}%')

buy&hold return: 799.38%
strategy return: 133.13%
buy&hold holding period: 3325 days
strategy holding period: 904 days
buy&hold daily return: 0.0661%
strategy daily return: 0.0937%
